# Custom GENO

In [1]:
import os

os.environ["ARCHDEFS"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/ARCHDefs/"
os.environ["SIFDECODE"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/SIFDecode/"
os.environ["MASTSIF"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/sif/"
os.environ["CUTEST"] = f"{os.path.abspath(os.getcwd())}/../CUTEst/CUTEst/"
os.environ["MYARCH"] = "mac64.osx.gfo"
os.environ["PYCUTEST_CACHE"] = (
    f"{os.path.abspath(os.getcwd())}/../pycutest_cache_holder/"
)

import pycutest

problemNames = pycutest.find_problems(constraints="unconstrained")
print(f"There are {len(problemNames)} unconstrained problems")

for problemName in problemNames[:2]:
    problem = pycutest.import_problem(problemName)
    problem.x0
    f, g = problem.obj(problem.x0, gradient=True)

There are 289 unconstrained problems


In [2]:
from timeit import default_timer as timer
import pycutest
import numpy as np

# import cupy as np  # uncomment this for GPU usage

In [3]:
def solve(
    problem: pycutest.CUTEstProblem,
    np,
    minimize,
    verbose: int = 0,
    line_search_debug_options=None,
):
    start = timer()
    options = {
        "eps_pg": 1e-5,  # Gradient for early stop
        "max_iter": 3000,
        "verbose": verbose,  # Set it to 0 to fully mute it.
        "max_ls": 30,
        "max_sample_count": 30,
    }

    if line_search_debug_options is not None:
        options["line_search_debug_options"] = line_search_debug_options

    fandg = lambda x: problem.obj(x, gradient=True)
    result = minimize(fandg, problem.x0, options=options, np=np)

    # assemble solution and map back to original problem
    elapsed = timer() - start
    report = {"result": result, "time": elapsed}
    return report

Run with eps_pg=1e-5, max_iter=3000, max_ls=30, max_sample_count=30

| Problem | Iteration | FunEvals | Time | Function Val | Proj Gradient | Comment |
| --- | --- | --- | --- | --- | --- | --- |
| SISSER | 1 | 3.9 | 473.68 | 1 | 1 |  comment |
| BOXPOWER | 0.96491 | 1.0681 | 18.565 | 9.5492 | 1.1824 |  comment |
| PALMER5C | 1 | 1.0714 | 413.45 | 1 | 1.0074 |  comment |
| DENSCHNB | 1 | 1 | 1.4505 | 1 | 1 |  comment |
| GBRAINLS | 0.58824 | 1.64 | 36.548 | 1 | 6.0943 |  comment |
| QING | 0.93056 | 3.3785 | 1147.6 | 7.0888 | 1.2127 |  comment |
| NONCVXU2 | 1 | 1.0091 | 1.6966 | 1 | 0.44979 | comment |
| DIXON3DQ | 1 | 0.99262 | 1.8006 | 1.0035 | 0.22406 | comment |
| BOX | 1 | 1.0833 | 2.8621 | 1 | 0.99929 | comment |
| BOX2 | 1 | 14.81 | 2415.4 | 1 | 1 | comment |
| BOX3 | 1 | 1.3846 | 114.52 | 0.56068 | 0.40995 | comment |
| NONDIA | 1 | 3.6364 | 84.712 | 1 | 1 | comment |
| DENSCHNC | 1 | 1 | 0.91546 | 1 | 1 | comment |
| LANCZOS3LS | 1.1622 | 2.8315 | 410.22 | 1.0354 | 0.34591 | comment |
| MGH09LS | 1.0263 | 1.2647 | 311.57 | 1 | 3.2315 | comment |
| LANCZOS1LS | 0.97333 | 1.5943 | 226 | 1.0048 | 0.82816 | comment |
| BA-L1LS | 0.89655 | 6.8498 | 2916.1 | 0.17307 | 0.76896 | Better value because of termination luck. Bad at small steps. |
| BROYDN7D | 0.99939 | 0.99176 | 1.802 | 1 | 0.96248 | |
| DENSCHNA | 1 | 1 | 1 | 1 | 1 | Randomly executes faster or slower than GENO |
| QUARTC | 1 | 1.6042 | 10.074 | 1 | 1 | |
| DEVGLA1 | 0.86667 | 0.6125 | 35.506 | 0.99957 | 0.33151 | |

### Skipped problems and reasons

| Problem | reason |
| --- | --- |
| CHWIRUT2LS | GENO stops with approx. error, while GP-GENO keeps going for a long time due to more samples beeing allowed |
| SPIN2LS | GP-GENO is exceptionally bad when each step is very small |
| LRCOVTYPE | Slow for both |
| MGH10LS | GENO stops with approx. error, while GP-GENO keeps going for a long time |
| TRIDIA | GP-GENO is exceptionally bad when each step is very small |
| MODBEALE | GP-GENO is exceptionally bad when each step is very small |
| BA-L16LS | Problem does not load at all |
| GAUSS1LS | GENO stops with approx. error, while GP-GENO keeps going for a long time |
| GAUSS3LS | GENO stops with approx. error, while GP-GENO keeps going for a long time |

In [4]:
problemNames = pycutest.find_problems(constraints="unconstrained")
problems_to_solve = [16]  # At most 288
problem_names_to_solve = np.array(problemNames)[problems_to_solve]
problem_names_to_solve = ["DEVGLA1"]

In [5]:
import bayesian_line_search.GPgenosolver as GPgenosolver
from bayesian_line_search.line_search import LineSearchDebugOptions
import genosolver

genosolver.check_version("0.1.0")

reports = dict()

for problemName in problem_names_to_solve:
    reports[problemName] = dict()
    problem = pycutest.import_problem(problemName)
    print(f"solving {problem.name} with geno")
    reports[problemName]["geno"] = solve(problem, np=np, minimize=genosolver.minimize)
    reports[problemName]["geno"]["report"] = problem.report()
    problem = pycutest.import_problem(problemName)
    print(f"solving {problem.name} with gp_geno")
    reports[problemName]["gp_geno"] = solve(
        problem, np=np, minimize=GPgenosolver.minimize
    )
    reports[problemName]["gp_geno"]["report"] = problem.report()

def print_solver_result(reports, solver):
    print(f"Results for {solver}:")
    print(f" {"Problem":<10} {"Iteration":<10} {"FunEvals":<10} {"Time":<10} {"Function Val":<15} {"Proj Gradient":<15}")
    for problem, solver_reports in reports.items():
        solver_report = solver_reports[solver]
        result = solver_report["result"]
        pg = np.linalg.norm(- result.jac, np.inf)
        print(f" {problem:<10} {result.nit:<10} {result.nfev:<10} {solver_report["time"]:<10.5g} {result.fun:<15.5e} {pg:<15.5e}")

def print_solver_comparison(reports, target, reference):
    print(f"Comparison between {target} and {reference}")
    print(f" {"Problem":<10} {"Iteration":<15} {"FunEvals":<15} {"Time":<15} {"Function Val":<15} {"Proj Gradient":<15}")
    for problem, solver_reports in reports.items():
        target_report = solver_reports[target]
        reference_report = solver_reports[reference]
        target_result = target_report["result"]
        reference_result = reference_report["result"]
        target_pg = np.linalg.norm(- target_result.jac, np.inf)
        reference_pg = np.linalg.norm(- reference_result.jac, np.inf)
        print(f" {problem:<10} {target_result.nit/reference_result.nit:<15.5g} {target_result.nfev/reference_result.nfev:<15.5g} {target_report["time"]/reference_report["time"]:<15.5g} {target_result.fun/reference_result.fun:<15.5g} {target_pg/reference_pg:<15.5g}")
        # For md:
        print(f"| {problem} | {target_result.nit/reference_result.nit:.5g} | {target_result.nfev/reference_result.nfev:.5g} | {target_report["time"]/reference_report["time"]:.5g} | {target_result.fun/reference_result.fun:.5g} | {target_pg/reference_pg:.5g} | |")

print_solver_result(reports, "geno")
print_solver_result(reports, "gp_geno")
print_solver_comparison(reports, "gp_geno", "geno")

solving DEVGLA1 with geno
solving DEVGLA1 with gp_geno


/Users/robinlabryga/.pyenv/versions/3.12.2/lib/python3.12/site-packages/genosolver/genosolver.py:106: RuntimeWarning: Option 'max_sample_count' is not supported.
  warnings.warn(f"Option '{opt}' is not supported.", RuntimeWarning)


Results for geno:
 Problem    Iteration  FunEvals   Time       Function Val    Proj Gradient  
 DEVGLA1    45         80         0.011309   2.12787e-13     6.52574e-06    
Results for gp_geno:
 Problem    Iteration  FunEvals   Time       Function Val    Proj Gradient  
 DEVGLA1    39         49         0.40152    2.12695e-13     2.16333e-06    
Comparison between gp_geno and geno
 Problem    Iteration       FunEvals        Time            Function Val    Proj Gradient  
 DEVGLA1    0.86667         0.6125          35.506          0.99957         0.33151        
| DEVGLA1 | 0.86667 | 0.6125 | 35.506 | 0.99957 | 0.33151 | |
